In [1]:
#!pip install huggingface_hub -q

In [7]:
#!pip install llama-cpp-python
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python

  Using cached llama_cpp_python-0.2.28-cp310-cp310-macosx_14_0_arm64.whl


In [8]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [9]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [10]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4,
    n_batch=1,
    n_gpu_layers=1,
    n_ctx=1024
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/jacqueslethuaut/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llam

In [30]:
def generate_llama_response(support_ticket_text):
  system_message = """
    [INST] «SYS»
    Introduction (System Role):
    I am the technical assistant designed to guide you in processing support tickets.
    Ticket Categorization:
    Your primary task is to categorize the support ticket into one of the following categories:
    -   Technical Issues
    -   Hardware Issues
    -   Data Recovery
    Response Options:
    Please respond with only one of the predefined categories listed above.
    Sub-Tasks:
    Once the category is identified, perform the following sub-tasks:
    -   Creating Tags: Create tags that will help further classify the ticket.
    -   Assigning Priority: Assign a priority level (e.g., "High" or "Low").
    -   Suggesting ETA: Provide an estimated time for resolving the issue.
    General Instructions:
    -   Categorization: Categorize the ticket only into the predefined categories.
    -   Reading Carefully: Read the support ticket text thoroughly and respond accordingly.
    Output Format:
    Please provide the response in the following structured format:
    Category: [Your Category Here]
    Tags: [Your Tags Here]
    Priority: [Your Priority Level Here]
    ETA: [Your Estimated Time Here]
    «/SYS» [/INST]
    """


  #  Combine user_prompt and system_message to create the prompt
  prompt = f"{support_ticket_text}\n{system_message}"

  #  Generate a response from the LLaMA model
  response = lcpp_llm(
    prompt=prompt,
    max_tokens=256,                  # Set the max tokens to generate
    temperature=0.0,                  # Set the temperature (between 0 and 1)
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop=['INST'],
    echo=False
  )
  #  Extract and return the response text
  response_text = response["choices"][0]["text"]
  return response_text


In [15]:
import pandas as pd


In [32]:
data = pd.read_csv('support_ticket_data.csv')

In [33]:
data.head()

,support_tick_id,support_ticket_text
0,ST2023-006,My internet connection has significantly slowe...
1,ST2023-007,Urgent help required! My laptop refuses to sta...
2,ST2023-008,I've accidentally deleted essential work docum...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...
4,ST2023-010,"My smartphone battery is draining rapidly, eve..."


In [34]:
data.shape

(21, 2)

In [35]:
data.isnull().sum()

support_tick_id        0
support_ticket_text    0
dtype: int64

In [36]:
data['llama_response'] = data['support_ticket_text'].apply(lambda x:generate_llama_response(x))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18500.48 ms
llama_print_timings:      sample time =      15.60 ms /   117 runs   (    0.13 ms per token,  7502.40 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   78682.81 ms /   476 runs   (  165.30 ms per token,     6.05 tokens per second)
llama_print_timings:       total time =   79626.70 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =   18500.48 ms
llama_print_timings:      sample time =       5.21 ms /    43 runs   (    0.12 ms per token,  8261.29 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   51972.27 ms /   401 runs   (  129.61 ms per token,     7.72 tokens per second)
llama_print_timings:       total time =   52

In [37]:
data.head()

,support_tick_id,support_ticket_text,llama_response
0,ST2023-006,My internet connection has significantly slowe...,"Sure, I'd be happy to help! Based on the info..."
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"Category: Hardware Issues\nTags: Laptop, Start..."
2,ST2023-008,I've accidentally deleted essential work docum...,"Sure, I can help you with that! Based on your..."
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"Sure, I can help you with that! Based on the ..."
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","Sure, I'd be happy to help! Based on the info..."


In [38]:
import json

In [39]:
def extract_json_data(response_text): 
    try:
        lines = response_text.strip().split('\n')
        response_dict = {line.split(':')[0].strip(): line.split(':')[1].strip() for line in lines if ':' in line}
        return response_dict
    except json.JSONDecodeError as e: 
        print(f"Error parsing JSON: {e}") 
        return {}

In [40]:
data['llama_response_parsed'] = data['llama_response'].apply(extract_json_data) 
data.head()

,support_tick_id,support_ticket_text,llama_response,llama_response_parsed
0,ST2023-006,My internet connection has significantly slowe...,"Sure, I'd be happy to help! Based on the info...","{'Category': 'Technical Issues', 'Tags': 'Inte..."
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"Category: Hardware Issues\nTags: Laptop, Start...","{'Category': 'Hardware Issues', 'Tags': 'Lapto..."
2,ST2023-008,I've accidentally deleted essential work docum...,"Sure, I can help you with that! Based on your...",{'Here are some sub-tasks to assist me in furt...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"Sure, I can help you with that! Based on the ...","{'Sure, I can help you with that! Based on the..."
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","Sure, I'd be happy to help! Based on the info...","{'Category': 'Technical Issues', 'Tags': 'Smar..."


In [42]:
def normalize_llama_response(response_dict):
    # Standardizing keys to lowercase
    normalized_response = {key.lower(): value.strip() for key, value in response_dict.items()}

    # Handling missing values
    for key in ['category', 'tags', 'priority', 'eta']:
        if key not in normalized_response:
            normalized_response[key] = None  # or any default value you prefer

    # Additional normalization can be added here as needed

    return normalized_response


In [44]:
data['llama_response_normalized'] = data['llama_response_parsed'].apply(normalize_llama_response)

print(data.head())

  support_tick_id                                support_ticket_text  \
0      ST2023-006  My internet connection has significantly slowe...   
1      ST2023-007  Urgent help required! My laptop refuses to sta...   
2      ST2023-008  I've accidentally deleted essential work docum...   
3      ST2023-009  Despite being in close proximity to my Wi-Fi r...   
4      ST2023-010  My smartphone battery is draining rapidly, eve...   

                                      llama_response  \
0   Sure, I'd be happy to help! Based on the info...   
1  Category: Hardware Issues\nTags: Laptop, Start...   
2   Sure, I can help you with that! Based on your...   
3   Sure, I can help you with that! Based on the ...   
4   Sure, I'd be happy to help! Based on the info...   

                               llama_response_parsed  \
0  {'Category': 'Technical Issues', 'Tags': 'Inte...   
1  {'Category': 'Hardware Issues', 'Tags': 'Lapto...   
2  {'Here are some sub-tasks to assist me in furt...   
3  {'S